In [1]:
# Load the .env file
#pip install -U python-dotenv
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# Cohere RAG pipeline

In [9]:
# def cohere_rag_answers(message):
#   import requests
#   import cohere
#   import json
#   import os
#   os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

#   # Define the API endpoint for streaming
#   url = "http://localhost:8000/v1/chat"
#   bearer = os.getenv('BEARER_SECRET_KEY')

#   # Set headers
#   headers = {
#       "User-Id": "me",
#       "Content-Type": "application/json",
#       "Authorization": f"Bearer {bearer}",
#       "Cohere-Stream": "true",  # Enable streaming for chatbot responses
#   }

#   # Set the message to send
#   #essage = "How can someone reach out to your team?"

#   # Create the payload as a JSON dictionary
#   data = {"message": message}

#   # Send the POST request using requests
#   response = requests.post(url, headers=headers, json=data)

#   # Check for successful response
#   if response.status_code == 200:
#     # Handle streaming response
#     for line in response.iter_lines():
#       # Decode the response (if necessary)
#       decoded_line = line.decode("utf-8")
#       # Process the received data from the stream (print it here)
#       response_data = json.loads(decoded_line)  # Parse the JSON string
#       print(decoded_line)
#   else:
#     print(f"Error: {response.status_code}")

#   return response_data

In [15]:
def cohere_rag_get_answers(message):
   import requests
   import cohere
   import json
   import os
   os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

   # Define the API endpoint for streaming
   url = "http://localhost:8000/v1/chat"
   bearer = os.getenv('BEARER_SECRET_KEY')

   # Set headers
   headers = {
       "User-Id": "me",
       "Content-Type": "application/json",
       "Authorization": f"Bearer {bearer}",
       "Cohere-Stream": "true",  # Enable streaming for chatbot responses
   }

   # Create the payload as a JSON dictionary
   data = {"message": message}

   # Send the POST request using requests
   response = requests.post(url, headers=headers, json=data)

   # Check for successful response
   if response.status_code == 200:
     # Handle streaming response
     for line in response.iter_lines():
       # Decode the response (if necessary)
       decoded_line = line.decode("utf-8")
       # Process the received data from the stream (print it here)
       response_data = json.loads(decoded_line)  # Parse the JSON string
       #print(decoded_line)
       return response_data.get("text")#, response_data.get("documents")
   else:
     print(f"Error: {response.status_code}")
     return ""

In [2]:
cohere_rag_get_answers("How can someone reach out to your team?")

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


{'answer': "You can use the following ways to reach out to my team:\n\n1. Email: You can send an email to the team's designated email address. Our email address is [team_email]. \n\n2, Telephone: You can also reach us via telephone. The number is [team_phone_number]. Our phone line is open during business hours. \n\n3. Social Media: We are active on various social media platforms. You can send us a direct message or comment on our posts with your queries or feedback. You may find us at [social media handle] on Facebook, [social media handle] on Twitter, and [social media handle] on Instagram. \n\n4. Website: You can submit an inquiry through our website. There is a 'Contact Us' form on the site that you can fill out, including your questions or comments. \n\n5. In Person: You are welcome to visit our physical location. Our office is at [team_address]. Our team members would be pleased to assist you during business hours. \n\nPlease note that response times may vary depending on the met

In [1]:
def cohere_rag_get_answers(message):
   import requests
   import cohere
   import json
   import os
   os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

   # Define the API endpoint for streaming
   url = "http://localhost:8000/v1/chat"
   bearer = os.getenv('BEARER_SECRET_KEY')

   # Set headers
   headers = {
       "User-Id": "me",
       "Content-Type": "application/json",
       "Authorization": f"Bearer {bearer}",
       "Cohere-Stream": "true",  # Enable streaming for chatbot responses
   }

   # Create the payload as a JSON dictionary
   data = {"message": message}

   # Send the POST request using requests
   response = requests.post(url, headers=headers, json=data)

   # Check for successful response
   if response.status_code == 200:
     # Handle streaming response
     for line in response.iter_lines():
       # Decode the response (if necessary)
       decoded_line = line.decode("utf-8")
       # Process the received data from the stream (print it here)
       response_data = json.loads(decoded_line)  # Parse the JSON string
       #print(decoded_line)
       return {
      'answer': response_data.get("text"),
      'contexts': response_data.get("documents")
      }
   else:
     print(f"Error: {response.status_code}")
     return {
       'answer': "Error: {response.status_code}",
      'contexts': []
     }

# RAG answer generation

In [17]:
import json

def test_answer_generation(json_file_path):
    """Appends generated text based on questions in a JSON file.

    Args:
        json_file_path (str): The path to the JSON file.

    Raises:
        FileNotFoundError: If the JSON file is not found.
    """

    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        for item in data:
            question = item['question']
            generated_text = cohere_rag_get_answers(question)  # Assuming generate_text returns a string
            item['response'] = generated_text

        with open(json_file_path, 'w') as f:
            json.dump(data, f, indent=4)

    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")


In [19]:
# Example usage:
from from_root import from_root
import os
folder = "data/test_dataset/test_dataset_hr.json"
json_file_path = os.path.join(from_root(), folder)
test_answer_generation(json_file_path)